In [3]:
import pandas as pd
import os
from datetime import datetime, timedelta
import requests
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from Alertlab_api.alertlab_api import get_token, get_locations, get_property_detailsv4, get_sensoreventsatlocation

# Constants
CLIENT_DATA_FILE = 'Client_data_processing/client_data.parquet'
LAST_UPDATE_FILE = 'Client_data_processing/last_updated.txt'
UPDATE_INTERVAL_DAYS = 30

In [7]:
token = '$2b$08$ajd9RlzullqvHiLPkJNfPuUS4Lb/zHj9SwqkOQ.F46rrfenLSeeeC'

In [8]:
locations = get_locations(token=token)

In [9]:
import pandas as pd
df = pd.DataFrame(locations)

In [10]:
df.head(2)

,_id,longitude,latitude,address,country,city,nodeType,name,parentID
0,1523553486229-16697,-75.730631,45.35481,"1343 Meadowlands Dr E, Ottawa, ON, Canada",Canada,Ottawa,building,Seneca,1732739642478-0990217076909
1,1523554162884-1322,-75.730008,45.35586,"1339 Meadowlands Dr E, Ottawa, ON, Canada",Canada,Ottawa,building,Huron,1732739642478-0990217076909


In [11]:
# Check if any parentID is also a location ID
parent_ids = df['parentID'].dropna().unique()
location_ids = df['_id'].unique()

# Find common IDs
common_ids = set(parent_ids).intersection(location_ids)

# Print the result
if common_ids:
    print("The following parentIDs are also location IDs:")
    print(common_ids)
else:
    print("No parentID is also a location ID.")

The following parentIDs are also location IDs:
{'1677779107294-0195245565449', '1738795775470-0799094208586', '1727964369529-1092897595040', '1702333342294-0922080745204', '1677780629661-0992963019910', '1677784054884-0155119629350', '1732658286294-0422248322880', '1735326705778-1004345064157', '1677784591137-0734492670227', '1677784451733-0465239304227', '1677785278077-0225827865281', '1677780662134-0088650130422', '1735257771958-0037165444517', '1677784836172-0176832585652', '1677780678538-1006867925362', '1677779085479-0507831866311', '1677785363504-0600186360468', '1677780437383-0806743464297', '1732048204321-0001998559583', '1725992552943-0216974921909', '1738424269254-0641538897382', '1735326418088-0254171339260', '1677784107269-0428937646393', '1677785325981-0133774059777', '1741784833281-0471260996965', '1739884161048-0170582997182', '1677784226857-0363602157483', '1661295154850-0890491971424'}


In [33]:
!pip install duckdb

   ---------------------------------------- 0.0/11.4 MB ? eta -:--:--
   ---------- ----------------------------- 2.9/11.4 MB 14.9 MB/s eta 0:00:01
   -------------- ------------------------- 4.2/11.4 MB 11.6 MB/s eta 0:00:01
   ------------------------ --------------- 7.1/11.4 MB 11.4 MB/s eta 0:00:01
   ------------------------------------ --- 10.5/11.4 MB 12.5 MB/s eta 0:00:01
   ---------------------------------------- 11.4/11.4 MB 12.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
# Filter the dataframe to include only rows with common_ids
parent_df = df[df['_id'].isin(common_ids)]

# Display the filtered dataframe
parent_df.head()

,_id,longitude,latitude,address,country,city,nodeType,name,parentID
20,1661295154850-0890491971424,NaN,NaN,NaN,NaN,NaN,org,Bob Langlois's Org,None
85,1677779085479-0507831866311,NaN,NaN,NaN,NaN,NaN,folder,NHD,1661295154850-0890491971424
86,1677779107294-0195245565449,NaN,NaN,NaN,NaN,NaN,folder,TMDL,1661295154850-0890491971424
88,1677780437383-0806743464297,NaN,NaN,NaN,NaN,NaN,folder,Palm Hospitality,1661295154850-0890491971424
90,1677780629661-0992963019910,NaN,NaN,NaN,NaN,NaN,folder,Aykler Real Estate,1661295154850-0890491971424


In [17]:
df_full = pd.merge(left=df, right=parent_df, on='_id', how='left')

In [18]:
df_full.head()

,_id,longitude_x,latitude_x,address_x,country_x,city_x,nodeType_x,name_x,parentID_x,longitude_y,latitude_y,address_y,country_y,city_y,nodeType_y,name_y,parentID_y
0,1523553486229-16697,-75.730631,45.354810,"1343 Meadowlands Dr E, Ottawa, ON, Canada",Canada,Ottawa,building,Seneca,1732739642478-0990217076909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1523554162884-1322,-75.730008,45.355860,"1339 Meadowlands Dr E, Ottawa, ON, Canada",Canada,Ottawa,building,Huron,1732739642478-0990217076909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1527247139149-20383,-79.363364,43.842100,"455 Cochrane Dr, Markham, ON, Canada",Canada,Markham,building,455 Cochrane,1657725892634-0639640771706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1528828823668-52545,-79.939006,43.649874,"110 Mowat Cres, Georgetown, ON, Canada",Canada,Georgetown,building,Base Res,1657725892634-0639640771706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1529065189195-80820,-79.363928,43.842691,"475 Cochrane Dr, Markham, ON, Canada",Canada,Markham,building,475 Cochrane,1657725892634-0639640771706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_full.dropna(how='all', axis=1, inplace=True)

In [28]:
df_full.isnull().sum()

_id              0
longitude_x     32
latitude_x      32
address_x       32
country_x       32
city_x          32
nodeType_x      21
name_x           0
parentID_x       1
nodeType_y     142
name_y         142
parentID_y     143
dtype: int64

In [29]:
df.shape

(170, 9)

In [32]:
df_full.dropna(axis=0,how='any',subset='name_y').nodeType_y.value_counts()

nodeType_y
folder    27
org        1
Name: count, dtype: int64

In [36]:
import duckdb

duckdb.query("SELECT * FROM df where _id = parentID").df() # None

# I want to test if any of the parentIDs are also presesnt as _ids.
common_ids = [item for item in common_ids]

In [46]:
df.head()

,_id,longitude,latitude,address,country,city,nodeType,name,parentID
0,1523553486229-16697,-75.730631,45.354810,"1343 Meadowlands Dr E, Ottawa, ON, Canada",Canada,Ottawa,building,Seneca,1732739642478-0990217076909
1,1523554162884-1322,-75.730008,45.355860,"1339 Meadowlands Dr E, Ottawa, ON, Canada",Canada,Ottawa,building,Huron,1732739642478-0990217076909
2,1527247139149-20383,-79.363364,43.842100,"455 Cochrane Dr, Markham, ON, Canada",Canada,Markham,building,455 Cochrane,1657725892634-0639640771706
3,1528828823668-52545,-79.939006,43.649874,"110 Mowat Cres, Georgetown, ON, Canada",Canada,Georgetown,building,Base Res,1657725892634-0639640771706
4,1529065189195-80820,-79.363928,43.842691,"475 Cochrane Dr, Markham, ON, Canada",Canada,Markham,building,475 Cochrane,1657725892634-0639640771706


In [70]:
# Perform self join to get parent name
result = pd.merge(df, df, left_on='parentID', right_on='_id', suffixes=('_child', '_parent'), how='left')

# Keep only relevant columns
result = result[['_id_child','parentID_child' ,'_id_parent', 'name_parent','name_child' ,'nodeType_parent', 'nodeType_child']]

# Count how many matched (i.e., parent name is not NaN)
matched_count = result['name_parent'].notna().sum()
total_with_parents = result['_id_parent'].notna().sum()

# Display missing parents
missing_parents = result[result['name_parent'].isna()][['_id_parent']].drop_duplicates()

# Display results
print("Matched Parent Names:", matched_count)
print("Total with ParentIDs:", total_with_parents)
print("Missing ParentIDs:\n", missing_parents)

Matched Parent Names: 146
Total with ParentIDs: 146
Missing ParentIDs:
   _id_parent
0        NaN


In [71]:
print(df.shape, result.shape)

(170, 9) (170, 7)


In [87]:
matching_properties = duckdb.query("SELECT * FROM result WHERE parentID_child = _id_parent").df()

In [89]:
matching_properties.name_parent.value_counts()

name_parent
Bob Langlois's Org           54
Q Residential                13
BGO Commercial               12
NHD                          11
Suncor                        6
King and Park                 6
BP Apartments                 5
TMDL                          4
BLH                           4
Wilson Blanchard              3
BGO Residential               3
Equity ICI                    2
Trusted Peak c/o Uniti PM     2
Yad Investments               2
Spring Living                 2
Li Limited                    2
ND Energy                     2
Onward Homes                  2
Campus Living                 2
Palm Hospitality              1
Astra                         1
Aykler Real Estate            1
Spicecart Living              1
Warrington PCI                1
BST Canada                    1
Novo Property Management      1
Northview REIT                1
Fengate Asset Management      1
Name: count, dtype: int64

In [81]:
orphan_properties = duckdb.query("SELECT * FROM result WHERE _id_parent is null").df()

In [86]:
orphan_properties

,_id_child,parentID_child,_id_parent,name_parent,name_child,nodeType_parent,nodeType_child
0,1523553486229-16697,1732739642478-0990217076909,None,None,Seneca,None,building
1,1523554162884-1322,1732739642478-0990217076909,None,None,Huron,None,building
2,1527247139149-20383,1657725892634-0639640771706,None,None,455 Cochrane,None,building
3,1528828823668-52545,1657725892634-0639640771706,None,None,Base Res,None,building
4,1529065189195-80820,1657725892634-0639640771706,None,None,475 Cochrane,None,building
5,1530099966432-16060,1657725892634-0639640771706,None,None,Armel DHW Res,None,building
6,1534177478759-60811,1719237958956-0102354844923,None,None,Norfinch,None,building
7,1534177774510-81712,1657725892634-0639640771706,None,None,Armel Main Res,None,building
8,1534957391420-30725,1657725892634-0639640771706,None,None,130 Res,None,building
9,1538563014511-95789,1657725892634-0639640771706,None,None,39 Parkcrest Res,None,building


In [93]:
result.to_csv('new_tombstone.csv')

### Full new tombstone 
Get all sensors 
Get all locations 
Merge all locations and sensors 
Get additional property details - numunits, numoccupents 

In [94]:
def get_all_sensors(token):

    url = f"https://api.alertaq.com/api/v4/public/sensors"
    headers = {"token": token}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch sensors: {response.text}")
    return response.json().get('dataModel', [])

sensors = get_all_sensors(token)
sensors


[{'_id': '343647043939373832003f00',
  'location_id': '1556881408563-0366253697556',
  'name': '7th Floor Sand Filter SE Platform',
  'serialNumber': 'LH0013798141',
  'friendlyType': 'Floodie'},
 {'_id': '343647043939373820002400',
  'location_id': '1556881408563-0366253697556',
  'name': 'Meter Room - Booster Pump',
  'serialNumber': 'LH0013795286',
  'friendlyType': 'Floodie'},
 {'_id': '37003a000847373336373936',
  'location_id': '1556881408563-0366253697556',
  'name': '10th Floor Cooling Tower Supply Meter',
  'serialNumber': 'WH0014045522',
  'friendlyType': 'Flowie'},
 {'_id': '3036470d3338303246004000',
  'location_id': '1556881408563-0366253697556',
  'name': '7th Floor Sand Filter N side platform',
  'serialNumber': 'LH0013730966',
  'friendlyType': 'Floodie'},
 {'_id': '310055000450483553353520',
  'location_id': '1557960406399-0012268381299',
  'name': '4 " Low Meter Bsmt',
  'serialNumber': 'WI0014190812',
  'friendlyType': 'Flowie'},
 {'_id': '3036470c3338303233004300',


In [109]:
sensor_df = pd.DataFrame(sensors)
sensor_df.head()

,_id,location_id,name,serialNumber,friendlyType
0,343647043939373832003f00,1556881408563-0366253697556,7th Floor Sand Filter SE Platform,LH0013798141,Floodie
1,343647043939373820002400,1556881408563-0366253697556,Meter Room - Booster Pump,LH0013795286,Floodie
2,37003a000847373336373936,1556881408563-0366253697556,10th Floor Cooling Tower Supply Meter,WH0014045522,Flowie
3,3036470d3338303246004000,1556881408563-0366253697556,7th Floor Sand Filter N side platform,LH0013730966,Floodie
4,310055000450483553353520,1557960406399-0012268381299,"4 "" Low Meter Bsmt",WI0014190812,Flowie


In [101]:
# Perform self join to get parent name
location_df = pd.merge(df, df, left_on='parentID', right_on='_id', suffixes=('_child', '_parent'), how='left')
location_df.head(2)

,_id_child,longitude_child,latitude_child,address_child,country_child,city_child,nodeType_child,name_child,parentID_child,_id_parent,longitude_parent,latitude_parent,address_parent,country_parent,city_parent,nodeType_parent,name_parent,parentID_parent
0,1523553486229-16697,-75.730631,45.35481,"1343 Meadowlands Dr E, Ottawa, ON, Canada",Canada,Ottawa,building,Seneca,1732739642478-0990217076909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1523554162884-1322,-75.730008,45.35586,"1339 Meadowlands Dr E, Ottawa, ON, Canada",Canada,Ottawa,building,Huron,1732739642478-0990217076909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
location_df = location_df[['_id_child', 'longitude_child', 'latitude_child', 'address_child','country_child', 'city_child', 'nodeType_child', 'name_child','parentID_child', '_id_parent', 'name_parent', 'nodeType_parent','parentID_parent']]
location_df.head()

,_id_child,longitude_child,latitude_child,address_child,country_child,city_child,nodeType_child,name_child,parentID_child,_id_parent,name_parent,nodeType_parent,parentID_parent
0,1523553486229-16697,-75.730631,45.354810,"1343 Meadowlands Dr E, Ottawa, ON, Canada",Canada,Ottawa,building,Seneca,1732739642478-0990217076909,NaN,NaN,NaN,NaN
1,1523554162884-1322,-75.730008,45.355860,"1339 Meadowlands Dr E, Ottawa, ON, Canada",Canada,Ottawa,building,Huron,1732739642478-0990217076909,NaN,NaN,NaN,NaN
2,1527247139149-20383,-79.363364,43.842100,"455 Cochrane Dr, Markham, ON, Canada",Canada,Markham,building,455 Cochrane,1657725892634-0639640771706,NaN,NaN,NaN,NaN
3,1528828823668-52545,-79.939006,43.649874,"110 Mowat Cres, Georgetown, ON, Canada",Canada,Georgetown,building,Base Res,1657725892634-0639640771706,NaN,NaN,NaN,NaN
4,1529065189195-80820,-79.363928,43.842691,"475 Cochrane Dr, Markham, ON, Canada",Canada,Markham,building,475 Cochrane,1657725892634-0639640771706,NaN,NaN,NaN,NaN


In [110]:
sensor_df = sensor_df[(sensor_df['friendlyType'] == 'Flowie-O') | (sensor_df['friendlyType'] == 'Flowie')]
sensor_df= sensor_df.groupby(by='location_id', as_index=False)[['_id', 'name', 'serialNumber', 'friendlyType']].agg(list)
sensor_df.head()

,location_id,_id,name,serialNumber,friendlyType
0,1523553486229-16697,"[2d0031000151363131363432, 4b002f0007513631303...","[Seneca High flow meter, Seneca Low flow meter]","[WE0010549758, WE0010547674]","[Flowie, Flowie]"
1,1523554162884-1322,[34001b000651363130333334],[Huron Single meter],[WE0010582181],[Flowie]
2,1527247139149-20383,[460046000751363234323834],[Portable],[WG0012390384],[Flowie]
3,1528828823668-52545,[250032000e47373136343738],[Basement],[WG0012584472],[Flowie]
4,1529065189195-80820,[5e003c000751363234323834],[475],[WG0012587565],[Flowie]


In [115]:
new_tombstone = pd.merge(location_df,sensor_df,left_on='_id_child', right_on='location_id', how='outer')
new_tombstone.to_csv('new_tombstone.csv')

### Cleaning Parent Dataframe


In [8]:
tombstone_df = pd.read_csv('E:/BondiWater/Bondi-Water/tombstoneunclean.csv')
tombstone_df.head(2)

,_id_child,longitude_child,latitude_child,address_child,country_child,city_child,nodeType_child,name_child,parentID_child,_id_parent,...,country_parent,city_parent,nodeType_parent,name_parent,parentID_parent,location_id,_id,name,serialNumber,friendlyType
0,1523553486229-16697,-75.730631,45.35481,"1343 Meadowlands Dr E, Ottawa, ON, Canada",Canada,Ottawa,building,Seneca,1732739642478-0990217076909,NaN,...,NaN,NaN,NaN,NaN,NaN,1523553486229-16697,"['2d0031000151363131363432', '4b002f0007513631...","['Seneca High flow meter', 'Seneca Low flow me...","['WE0010549758', 'WE0010547674']","['Flowie', 'Flowie']"
1,1523554162884-1322,-75.730008,45.35586,"1339 Meadowlands Dr E, Ottawa, ON, Canada",Canada,Ottawa,building,Huron,1732739642478-0990217076909,NaN,...,NaN,NaN,NaN,NaN,NaN,1523554162884-1322,['34001b000651363130333334'],['Huron Single meter'],['WE0010582181'],['Flowie']


In [22]:
df = tombstone_df.copy() 

# 2. Remove folder-type children
df = df[(df['nodeType_child'] != 'folder') & (df['nodeType_child'] != 'org')]

df.tail(2)
# Set to Other when both location and sensors exist

,_id_child,longitude_child,latitude_child,address_child,country_child,city_child,nodeType_child,name_child,parentID_child,_id_parent,...,country_parent,city_parent,nodeType_parent,name_parent,parentID_parent,location_id,_id,name,serialNumber,friendlyType
136,1742496666519-0354822857628,-75.684894,45.432625,"345 Barber St, Ottawa, ON K1N 5R5, Canada",Canada,Ottawa,building,345-347 Barber St Res,1732658286294-0422248322880,1732658286294-0422248322880,...,NaN,NaN,folder,Yad Investments,1661295154850-0890491971424,1742496666519-0354822857628,['310037000547373336373936'],['WI0016235173'],['WI0016235173'],['Flowie']
137,1742505591205-0650659329656,-75.843123,45.317312,"1 Mill Hill Rd, Ottawa, ON K2H 9L6, Canada",Canada,Ottawa,building,Robertson House Res,1735257771958-0037165444517,1735257771958-0037165444517,...,NaN,NaN,folder,Spring Living,1661295154850-0890491971424,1742505591205-0650659329656,['33324703383031343e004400'],['EI0019036093'],['EI0019036093'],['Flowie-O']


In [23]:
df = df.dropna(subset=['name_child'])
df.tail(2)

,_id_child,longitude_child,latitude_child,address_child,country_child,city_child,nodeType_child,name_child,parentID_child,_id_parent,...,country_parent,city_parent,nodeType_parent,name_parent,parentID_parent,location_id,_id,name,serialNumber,friendlyType
136,1742496666519-0354822857628,-75.684894,45.432625,"345 Barber St, Ottawa, ON K1N 5R5, Canada",Canada,Ottawa,building,345-347 Barber St Res,1732658286294-0422248322880,1732658286294-0422248322880,...,NaN,NaN,folder,Yad Investments,1661295154850-0890491971424,1742496666519-0354822857628,['310037000547373336373936'],['WI0016235173'],['WI0016235173'],['Flowie']
137,1742505591205-0650659329656,-75.843123,45.317312,"1 Mill Hill Rd, Ottawa, ON K2H 9L6, Canada",Canada,Ottawa,building,Robertson House Res,1735257771958-0037165444517,1735257771958-0037165444517,...,NaN,NaN,folder,Spring Living,1661295154850-0890491971424,1742505591205-0650659329656,['33324703383031343e004400'],['EI0019036093'],['EI0019036093'],['Flowie-O']


In [24]:
df.loc[df['nodeType_child'].isna() & df['serialNumber'].notna(), 'nodeType_child'] = 'building'

In [ ]:
df[df['_id_child']=='1673889116124-0983697005124']
#node_type building correct

,_id_child,longitude_child,latitude_child,address_child,country_child,city_child,nodeType_child,name_child,parentID_child,_id_parent,...,country_parent,city_parent,nodeType_parent,name_parent,parentID_parent,location_id,_id,name,serialNumber,friendlyType
79,1673889116124-0983697005124,-79.429291,43.635352,"35 Spencer Ave, Toronto, ON, Canada",Canada,Toronto,building,35 Spencer Res,1657725892634-0639640771706,NaN,...,NaN,NaN,NaN,NaN,NaN,1673889116124-0983697005124,['3e002f000447373336373936'],['Spencer main'],['WJ0016488845'],['Flowie']


In [26]:
df = df.dropna(subset=['_id', 'serialNumber'], how='all')

In [27]:
df['name_parent'] = df['name_parent'].replace(
    "Bob Langlois's Org", 'Other', regex=False
)

In [28]:
df.name_parent.value_counts()

name_parent
Other                        20
Q Residential                12
BGO Commercial               11
NHD                          11
Suncor                        6
King and Park                 6
BP Apartments                 5
BLH                           4
TMDL                          4
Yad Investments               4
Wilson Blanchard              3
BGO Residential               3
Spring Living                 3
Boardwalk                     2
Onward Homes                  2
Trusted Peak c/o Uniti PM     2
Campus Living                 2
Equity ICI                    2
Li Limited                    2
ND Energy                     2
Palm Hospitality              1
Spicecart Living              1
Astra                         1
Aykler Real Estate            1
BST Canada                    1
Warrington PCI                1
Northview REIT                1
Novo Property Management      1
Fengate Asset Management      1
Name: count, dtype: int64

In [37]:
df.loc[df['_id_child'].notna() & df['serialNumber'].notna() & df['name_parent'].isna(), 'name_parent'] = 'Other'
df.to_csv('tombstone.csv')

In [30]:
df.name_parent.value_counts()

name_parent
Other                        43
Q Residential                12
BGO Commercial               11
NHD                          11
Suncor                        6
King and Park                 6
BP Apartments                 5
BLH                           4
TMDL                          4
Yad Investments               4
Wilson Blanchard              3
BGO Residential               3
Spring Living                 3
Boardwalk                     2
Onward Homes                  2
Trusted Peak c/o Uniti PM     2
Campus Living                 2
Equity ICI                    2
Li Limited                    2
ND Energy                     2
Palm Hospitality              1
Spicecart Living              1
Astra                         1
Aykler Real Estate            1
BST Canada                    1
Warrington PCI                1
Northview REIT                1
Novo Property Management      1
Fengate Asset Management      1
Name: count, dtype: int64

### Enriching the property number of suites. 
Since all 221 sensors are populated

We use this workflow after the query button is clicked to get property details just for filtered data. This way we don't get exposed in their backed for getting extra data 

In [1]:
import os
import sys
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from Alertlab_api.alertlab_api import get_token, get_property_detailsv4

df = pd.read_csv('new_tombstone.csv')

2025-03-26 15:17:52,034 - Alertlab_api.aws_utils - INFO - Problem with hidden API response code


In [2]:
test_property = df['_id_child'].iloc[1]
test_property

'1523554162884-1322'

In [3]:
test_property_details = get_property_detailsv4(test_property)

2025-03-26 15:22:43,015 - Alertlab_api.aws_utils - INFO - Problem with hidden API response code


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [47]:
test_property_details

{'dataModel': [{'_id': '1523553486229-16697',
   'timezone': 'America/Toronto',
   'postalCode': 'K2E 6K6',
   'longitude': -75.7306314,
   'latitude': 45.3548104,
   'vacationMode': False,
   'unoccupiedSuites': None,
   'numberSuites': 251,
   'commercialPropertyType': 'multi-residential',
   'waterBillingEveryXMonths': 1,
   'waterBillingStartDate': '2018-04-27T04:00:00.000Z',
   'address': '1343 Meadowlands Dr E, Ottawa, ON, Canada',
   'numOccupants': None,
   'occupantType': 3,
   'buildingType': None,
   'age': 7,
   'size': 5000,
   'country': 'Canada',
   'province': 'ON',
   'city': 'Ottawa',
   'street': '1343 Meadowlands Dr E',
   'owner': '1513867260462-24600',
   'users': ['1538056346763-33158',
    '1559134317537-0639225134191',
    '1603196375876-0100568571260',
    '1611950231283-0659799125179',
    '1550079886299-0580342988343'],
   'ancestors': ['1661902227606-1051162043177', '1732739642478-0990217076909'],
   'nodeType': 'building',
   'addressType': 'validated',
  

In [4]:
test_property_details = {'dataModel': [{'_id': '1523554162884-1322', 'timezone': 'America/Toronto', 'postalCode': 'K2E 7B4', 'longitude': -75.7300076, 'latitude': 45.3558603, 'vacationMode': False, 'unoccupiedSuites': None, 'numberSuites': 251, 'commercialPropertyType': 'multi-residential', 'waterBillingEveryXMonths': 1, 'waterBillingStartDate': '2018-04-27T04:00:00.000Z', 'address': '1339 Meadowlands Dr E, Ottawa, ON, Canada', 'numOccupants': None, 'occupantType': 3, 'buildingType': None, 'age': 7, 'size': None, 'country': 'Canada', 'province': 'ON', 'city': 'Ottawa', 'street': '1339 Meadowlands Dr E', 'owner': '1513867260462-24600', 'users': ['1538056346763-33158', '1559134317537-0639225134191', '1603196375876-0100568571260', '1611950231283-0659799125179', '1542911668336-0673976150469', '1550079886299-0580342988343'], 'ancestors': ['1661902227606-1051162043177', '1732739642478-0990217076909'], 'nodeType': 'building', 'addressType': 'validated', 'numberFloors': None, 'parentIDs': ['1680489310344-1078075703538'], 'sensors': [{'_id': '34001b000651363130333334', 'location_id': '1523554162884-1322'}], 'name': 'Huron', 'hasNotes': None}], 'error': None}

In [7]:
import json
json.dumps(test_property_details)

'{"dataModel": [{"_id": "1523554162884-1322", "timezone": "America/Toronto", "postalCode": "K2E 7B4", "longitude": -75.7300076, "latitude": 45.3558603, "vacationMode": false, "unoccupiedSuites": null, "numberSuites": 251, "commercialPropertyType": "multi-residential", "waterBillingEveryXMonths": 1, "waterBillingStartDate": "2018-04-27T04:00:00.000Z", "address": "1339 Meadowlands Dr E, Ottawa, ON, Canada", "numOccupants": null, "occupantType": 3, "buildingType": null, "age": 7, "size": null, "country": "Canada", "province": "ON", "city": "Ottawa", "street": "1339 Meadowlands Dr E", "owner": "1513867260462-24600", "users": ["1538056346763-33158", "1559134317537-0639225134191", "1603196375876-0100568571260", "1611950231283-0659799125179", "1542911668336-0673976150469", "1550079886299-0580342988343"], "ancestors": ["1661902227606-1051162043177", "1732739642478-0990217076909"], "nodeType": "building", "addressType": "validated", "numberFloors": null, "parentIDs": ["1680489310344-10780757035

In [10]:
test_property_details['dataModel'][0]['numberSuites']

251

In [12]:
number_of_suites = test_property_details['dataModel'][0]['numberSuites']
if number_of_suites is None: 
    number_of_suites = 1
number_of_floors = test_property_details['dataModel'][0]['numberFloors']
if number_of_floors is None:
    number_of_floors = 1
CommercialPropertyType = test_property_details['dataModel'][0]['commercialPropertyType']
property_age = test_property_details['dataModel'][0]['age']
number_of_user = len(test_property_details['dataModel'][0]['users'])

In [13]:
number_of_user

6

In [11]:
property_details_df = pd.DataFrame.from_dict(test_property_details['dataModel'])

In [57]:
property_details_df.head(1)

,_id,timezone,postalCode,longitude,latitude,vacationMode,unoccupiedSuites,numberSuites,commercialPropertyType,waterBillingEveryXMonths,...,owner,users,ancestors,nodeType,addressType,numberFloors,parentIDs,sensors,name,hasNotes
0,1523553486229-16697,America/Toronto,K2E 6K6,-75.730631,45.35481,False,None,251,multi-residential,1,...,1513867260462-24600,"[1538056346763-33158, 1559134317537-0639225134...","[1661902227606-1051162043177, 1732739642478-09...",building,validated,None,[1680489310344-1078075703538],"[{'_id': '2d0031000151363131363432', 'location...",Seneca,None


In [59]:
property_details_df.dtypes

_id                          object
timezone                     object
postalCode                   object
longitude                   float64
latitude                    float64
vacationMode                   bool
unoccupiedSuites             object
numberSuites                  int64
commercialPropertyType       object
waterBillingEveryXMonths      int64
waterBillingStartDate        object
address                      object
numOccupants                 object
occupantType                  int64
buildingType                 object
age                           int64
size                          int64
country                      object
province                     object
city                         object
street                       object
owner                        object
users                        object
ancestors                    object
nodeType                     object
addressType                  object
numberFloors                 object
parentIDs                   

In [12]:
for col in property_details_df.columns:
    print(col, property_details_df[col].iloc[0])

_id 1523553486229-16697
timezone America/Toronto
postalCode K2E 6K6
longitude -75.7306314
latitude 45.3548104
vacationMode False
unoccupiedSuites None
numberSuites 251
commercialPropertyType multi-residential
waterBillingEveryXMonths 1
waterBillingStartDate 2018-04-27T04:00:00.000Z
address 1343 Meadowlands Dr E, Ottawa, ON, Canada
numOccupants None
occupantType 3
buildingType None
age 7
size 5000
country Canada
province ON
city Ottawa
street 1343 Meadowlands Dr E
owner 1513867260462-24600
users ['1538056346763-33158', '1559134317537-0639225134191', '1603196375876-0100568571260', '1611950231283-0659799125179', '1550079886299-0580342988343']
ancestors ['1661902227606-1051162043177', '1732739642478-0990217076909']
nodeType building
addressType validated
numberFloors None
parentIDs ['1680489310344-1078075703538']
sensors [{'_id': '2d0031000151363131363432', 'location_id': '1523553486229-16697'}, {'_id': '4b002f000751363130333334', 'location_id': '1523553486229-16697'}]
name Seneca
hasNotes

- Number of floors
- Size
- Commercial Property type
- Number of suites
- Number of floors

### Testing the timeseries list function

In [ ]:
import requests 
import json
import ast 

token = '$2b$08$ajd9RlzullqvHiLPkJNfPuUS4Lb/zHj9SwqkOQ.F46rrfenLSeeeC'
#Change to v4 and add help comments for rate, and series. 
def _get_timeseries(sensor_id, start_date, end_date, rate="h", series="W", token=None):
    """Fetch timeseries data for a sensor."""
    if not token:
        token = get_token()
    if not sensor_id or not start_date or not end_date:
        raise ValueError("sensor_id, start_date, and end_date are required")
    timeseriesurl = f"https://api.alertaq.com/api/v4/public/timeseries?from={start_date}&to={end_date}&rate={rate}&series={series}&sensorID={sensor_id}"

    response = requests.get(timeseriesurl, headers={"Token": token})
    timeseries = response.json()
    print(timeseries)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch timeseries: {response.text}")
    response_json = response.json()
    values = timeserieslist[0]['dataModel'][list[0]]
    df = pd.DataFrame(values, columns=['time', 'series'])
    df['Datetime'] = pd.to_datetime(df['time'], unit='ms') - timedelta(hours=4)
    return df

def get_list_timeseries(sensor_list, start_date="1742810400", end_date="1742832000", rate="h", series="W"):
    time_series_list = []
    for sensor in ast.literal_eval(sensor_list):
        time_series_data = _get_timeseries(sensor_id=sensor,
                                             start_date=start_date,
                                             end_date=end_date,
                                             rate=rate,
                                             token = token
                                            )
        if time_series_data is not None:
            time_series_list.append(time_series_data)
    return time_series_list

In [5]:
import pandas as pd
df = pd.read_csv('new_tombstone.csv')

In [4]:
df.head()

,_id_child,longitude_child,latitude_child,address_child,country_child,city_child,nodeType_child,name_child,parentID_child,_id_parent,...,country_parent,city_parent,nodeType_parent,name_parent,parentID_parent,location_id,_id,name,serialNumber,friendlyType
0,1523553486229-16697,-75.730631,45.354810,"1343 Meadowlands Dr E, Ottawa, ON, Canada",Canada,Ottawa,building,Seneca,1732739642478-0990217076909,NaN,...,NaN,NaN,NaN,Other,NaN,1523553486229-16697,"['2d0031000151363131363432', '4b002f0007513631...","['Seneca High flow meter', 'Seneca Low flow me...","['WE0010549758', 'WE0010547674']","['Flowie', 'Flowie']"
1,1523554162884-1322,-75.730008,45.355860,"1339 Meadowlands Dr E, Ottawa, ON, Canada",Canada,Ottawa,building,Huron,1732739642478-0990217076909,NaN,...,NaN,NaN,NaN,Other,NaN,1523554162884-1322,['34001b000651363130333334'],['Huron Single meter'],['WE0010582181'],['Flowie']
2,1527247139149-20383,-79.363364,43.842100,"455 Cochrane Dr, Markham, ON, Canada",Canada,Markham,building,455 Cochrane,1657725892634-0639640771706,NaN,...,NaN,NaN,NaN,Other,NaN,1527247139149-20383,['460046000751363234323834'],['Portable'],['WG0012390384'],['Flowie']
3,1528828823668-52545,-79.939006,43.649874,"110 Mowat Cres, Georgetown, ON, Canada",Canada,Georgetown,building,Base Res,1657725892634-0639640771706,NaN,...,NaN,NaN,NaN,Other,NaN,1528828823668-52545,['250032000e47373136343738'],['Basement'],['WG0012584472'],['Flowie']
4,1529065189195-80820,-79.363928,43.842691,"475 Cochrane Dr, Markham, ON, Canada",Canada,Markham,building,475 Cochrane,1657725892634-0639640771706,NaN,...,NaN,NaN,NaN,Other,NaN,1529065189195-80820,['5e003c000751363234323834'],['475'],['WG0012587565'],['Flowie']


In [6]:
import ast 
test_ids = df['_id'].iloc[16]
list = ast.literal_eval(test_ids)

for id in list:
    print(id)
    

37003a000847373336373936


In [55]:
type(list)

list

In [7]:
timeserieslist = get_list_timeseries(test_ids)

{'error': None, 'dataModel': {'37003a000847373336373936': [[1742810400000, 941.5262636273537], [1742814000000, 396.4321110009911], [1742817600000, 247.7700693756194], [1742821200000, 426.1645193260654], [1742824800000, 545.0941526263628], [1742828400000, 346.87809712586716]]}}


In [24]:
values = timeserieslist[0]['dataModel'][list[0]]
df1 = pd.DataFrame(values, columns=['time', 'series'])
df1['datetime'] = pd.to_datetime(df1['time'], unit='ms') - timedelta(hours=4)
df1.head()

,time,series,datetime
0,1742810400000,941.526264,2025-03-24 06:00:00
1,1742814000000,396.432111,2025-03-24 07:00:00
2,1742817600000,247.770069,2025-03-24 08:00:00
3,1742821200000,426.164519,2025-03-24 09:00:00
4,1742824800000,545.094153,2025-03-24 10:00:00


In [ ]:
from datetime import timedelta

df1['datetime'] = pd.to_datetime(df1['time'], unit='ms') - timedelta(hours=4)

In [23]:
df1.head()

,time,series,datetime
0,1742810400000,941.526264,2025-03-24 06:00:00
1,1742814000000,396.432111,2025-03-24 07:00:00
2,1742817600000,247.770069,2025-03-24 08:00:00
3,1742821200000,426.164519,2025-03-24 09:00:00
4,1742824800000,545.094153,2025-03-24 10:00:00


In [45]:
timeserieslist[0]

{'error': None, 'dataModel': []}

In [23]:
json

'[{"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}, {"error": null, "dataModel": []}]'

In [1]:
import pandas as pd

df = pd.read_csv('new_tombstone.csv')

In [2]:
df.columns

Index(['_id_child', 'longitude_child', 'latitude_child', 'address_child',
       'country_child', 'city_child', 'nodeType_child', 'name_child',
       'parentID_child', '_id_parent', 'longitude_parent', 'latitude_parent',
       'address_parent', 'country_parent', 'city_parent', 'nodeType_parent',
       'name_parent', 'parentID_parent', 'location_id', '_id', 'name',
       'serialNumber', 'friendlyType'],
      dtype='object')

In [ ]:
renamed_columns = {'_id': 'sensor_ids', 
           'name' : 'sensor_names',
           'serialNumber': 'sensor_serialNumbers', 
           'friendlyType': 'sensor_friendlyType'}

df.rename(columns=renamed_columns, inplace=True)
df.columns

Index(['_id_child', 'longitude_child', 'latitude_child', 'address_child',
       'country_child', 'city_child', 'nodeType_child', 'name_child',
       'parentID_child', '_id_parent', 'longitude_parent', 'latitude_parent',
       'address_parent', 'country_parent', 'city_parent', 'nodeType_parent',
       'name_parent', 'parentID_parent', 'location_id', 'sensor_ids',
       'sensor_names', 'sensor_serialNumber', 'sensor_friendlyType'],
      dtype='object')